<a href="https://colab.research.google.com/github/szkjiro/program/blob/main/PyAozora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自然言語処理をする

ワードクラウドと呼ばれる，ツイッター（現X）などで使われる単語頻度を直感的に表示する手法にワードクラウドがあります．
例：[国会議案データでワードクラウドを作ってみよう](https://www.mediatechnology.jp/entry/parliamentary-bills-database)

このワードクラウドを作成するには，
1. もとの文章を単語単位に分ける
2. 各単語の頻度を集計する
3. ワードクラウドに作成する

というステップを踏みます．
単語単位に分ける部分（形態素解析）は，自然言語処理技術としても触れます．

この演習課題では，単に頻度集計の一例としてワードクラウドを体験することに集中します．

統計処理としては単に集計するだけなのできわめて単純です．
新しい技術，最近よく見かける技術だからと言って，基礎的なところに注目してみれば，最新の難解な技術が使われているわけではない一例です．

## Pythonで形態素解析を使う

最初に形態素解析ツールMeCab[参考：MeCabのサイト](https://taku910.github.io/mecab/)を使えるようにします．
ワードクラウドで使うためのフォントもインストールしておきます．

In [ ]:
!apt-get -y install fonts-ipafont-gothic
! pip install mecab-python3 unidic-lite

次に形態素解析が実際にうまくできるか，簡単なサンプルで試します．

In [ ]:
import MeCab
print(MeCab.Tagger().parse("失敗に挑戦しないことが最大の失敗である"))

上の実行例でわかるように，形態素解析では入力データを単語単位に分割します．
次の例では上の文を分かち書きだけにして表示しました．
中学校国語の文法で，こうした単語の分割を学習した人もいるのではありませんか．

In [3]:
print(MeCab.Tagger("-Owakati").parse("失敗に挑戦しないことが最大の失敗である"))

失敗 に 挑戦 し ない こと が 最大 の 失敗 で ある 



## 青空文庫からテキストファイルを取得する

ここでは青空文庫から作品ファイルを取得してみます．
以上の各例題が実行済みでないと，この先はうまく進まないので注意しましょう．
以下の例題の実行が済んだら，
みなさんも好みの作品のURLで置き換えて実行してみてください．

ファイル取得用のURLは，[青空文庫](https://www.aozora.gr.jp)にて，作品を選択すると以下が表示されます．

>［ファイルのダウンロード｜いますぐXHTML版で読む］

この右側の「いますぐXHTML版で読む」を選択すれば，作品が表示されます．
その作品のリンクをコピーして，以下のプログラムの所定の部分を置き換えてください．
以下の例題で，この変更をするものは，以下の2つのプログラムだけです．

まずは青空文庫からあなたのGoogle Colab環境にファイルをダウンロードします．

In [ ]:
! wget https://www.aozora.gr.jp/cards/000042/files/1108_13798.html

次のプログラムでは上でダウンロードしたファイルを使えるようにします．
そのために上のプログラムで取得するファイル名（各作品に対応）を変更した場合，以下でも同じファイル名に変更しておいてください．
今回の課題で書き換えを行うのはここまでです．

ダウンロードしたファイルはHTMLというウェブページ用の形式であるため，本文以外の情報（HTMLタグなど）を多く含みます．
それらの情報は文章の分析に不要なので，BeautifulSoupというツールを用いて除去します．

In [ ]:
import re
f = open('1108_13798.html', 'r', encoding='SJIS')
html_file = f.read()
f.close()
# 代入結果の確認
html_file
# 本文のみにする
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_file, 'lxml')

main_text = soup.find("div", class_='main_text')

#ルビが振ってあるのを削除
for yomigana in main_text.find_all(["rp","h4","rt"]):
  yomigana.decompose()

bunko_text = [line.strip() for line in main_text.text.strip().splitlines()]

print(bunko_text)

青空文庫では，他にも各作品を作成したボランティア情報など，作品そのものの分析に不要な情報も含みますが，ここでは処理を簡単に見せるため省略しています．

## ワードクラウドの出力

以下のプログラムは，ワードクラウドに出力する準備状況の確認に，すでに試した分かち書きにします．

In [ ]:
bunko_text2=','.join(bunko_text)
bunko_text3 = MeCab.Tagger("-Owakati").parse(bunko_text2)
print(bunko_text3)

以下ではワードクラウドを出力するためのライブラリをインストールし，
上で準備した文章を与えて出力しています．

In [ ]:
!pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# 文字表示用フォント指定
f_path = '/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf' #日本語フォントの取得
wordcloud = WordCloud(background_color="white", font_path=f_path, width=700, height=600)
wordcloud.generate(bunko_text3)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud)
plt.axis("off") #メモリの非表示
# plt.show()
# 以下のファイル名で保存されます．提出用はダウンロードして利用してください．
plt.savefig("word_cloud1.png")
from google.colab import files
files.download("word_cloud1.png")

上の例では「は，の，に，を」を含めたすべての単語を集計しています．
そのため，ワードクラウドにはそれらが一番頻出する単語として表示されました．
次では名詞のみに絞ってワードクラウドを作成するための処理をしています．

In [ ]:
import MeCab
# 取り出したい品詞
select_conditions = ['名詞']
# select_conditions = ['形容詞', '副詞']

# 分かち書きオブジェクト
tagger = MeCab.Tagger('')
tagger.parse('')

# 関数の定義
# 指定した品詞を取り出し分かち書きの文を返す
def wakati_text(text):
    # 分けてノードごとにする
    node = tagger.parseToNode(text)
    terms = []

    while node:
        # 単語
        term = node.surface
        # 品詞
        pos = node.feature.split(',')[0]
        # もし品詞が条件と一致してたら
        if pos in select_conditions:
            terms.append(term)
        node = node.next

    # 連結する
    text_result = ' '.join(terms)
    return text_result

bunko_text2=','.join(bunko_text)
bunko_text3 = MeCab.Tagger("-Owakati").parse(bunko_text2)
bunko_text4 = wakati_text(bunko_text3)

from wordcloud import WordCloud
import matplotlib.pyplot as plt
# 文字表示用フォント指定
f_path = '/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf' #日本語フォントの取得
wordcloud = WordCloud(background_color="white", font_path=f_path, width=700, height=600)
wordcloud.generate(bunko_text4)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud)
plt.axis("off") #メモリの非表示
# plt.show()
# 以下のファイル名で保存されます．提出用はダウンロードして利用してください．
# ファイル名を実行ごとに変更すれば別々のファイルに保存できます
# 拡張子は変更しないでください
plt.savefig("word_cloud1.png")
from google.colab import files
files.download("word_cloud1.png")

各自で分析したワードクラウドの提出について．この演習ファイルを利用している授業によっては，自身で分析させたワードクラウドの表示結果ファイルを提出するよう指示があると思います．
該当する人は，Google Colabのフォルダ内に画像ファイル（拡張子png）が出来上がっているので，そのファイルを各自でダウンロードの上，提出してください．